<a href="https://colab.research.google.com/github/dsynderg/CS-479-machine-translation/blob/main/my_mtw_sentancepiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download data
!wget https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
!tar xf toy-ende.tar.gz

--2025-10-07 22:36:07--  https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.47.6, 54.231.196.104, 52.216.249.182, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.47.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1662081 (1.6M) [application/x-gzip]
Saving to: ‘toy-ende.tar.gz’

toy-ende.tar.gz     100%[===================>]   1.58M  1.18MB/s    in 1.3s    

2025-10-07 22:36:09 (1.18 MB/s) - ‘toy-ende.tar.gz’ saved [1662081/1662081]



In [2]:
!pip install sentencepiece

In [4]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
  input='src_train.txt',
  model_prefix='english_sp_model',
  vocab_size=16000,
)

spm.SentencePieceTrainer.train(
  input='tgt_train.txt',
  model_prefix='spanish_sp_model',
  vocab_size=16000,
)

In [ ]:
!pip install OpenNMT-py
!pip install "numpy<2.0" # This fixes an error caused by OpenNMT not being maintained

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 144.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 130.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Build YAML Config file
yaml_text = """
# toy_en_de.yaml

## Where the samples will be written
save_data: example

src_vocab: example.vocab.src
tgt_vocab: example.vocab.tgt

# Prevent overwriting existing files in the folder
overwrite: True

# -- Sentencepiece Params --
# Tokenization options
src_subword_type: sentencepiece
src_subword_model: english_sp_model.model
tgt_subword_type: sentencepiece
tgt_subword_model: spanish_sp_model.model

# Number of candidates for SentencePiece sampling
subword_nbest: 20
# Smoothing parameter for SentencePiece sampling
subword_alpha: 0.1
# Specific arguments for pyonmttok
src_onmttok_kwargs: "{'mode': 'none', 'spacer_annotate': True}"
tgt_onmttok_kwargs: "{'mode': 'none', 'spacer_annotate': True}"


# Corpus opts:
data:
    corpus_1:
        path_src: src_train.txt
        path_tgt: tgt_train.txt
        transforms: [onmt_tokenize]
    valid:
        path_src: src_val.txt
        path_tgt: tgt_val.txt
        transforms: [onmt_tokenize]

# Train on a single GPU
bucket_size: 10000
world_size: 1
gpu_ranks: [0]

# Where to save the checkpoints
save_model: toy-ende/run/model
save_checkpoint_steps: 500
train_steps: 20000
valid_steps: 500
"""

with open("sentance_peice_yaml.yaml", "w") as f:
    f.write(yaml_text)